In [ ]:
#@title # 🛠️ ComfyUI Configuration Panel
#@markdown ---
#@markdown ### 📁 Storage Management (Google Drive)
#@markdown Choose how you want to handle your files. Mounting Drive allows your models and results to persist after closing Colab.
MODO_DRIVE = "SOLO_LOCAL" #@param ["MONTAR", "DESMONTAR", "SOLO_LOCAL"]
USAR_DRIVE_PARA_COMFY = True #@param {type:"boolean"}
#@markdown > **Note:** If you check the box above, ComfyUI will be installed in `/content/drive/MyDrive/AI/ComfyUI`. This saves time in future sessions as you won't have to re-download everything.
RUTA_PERSONALIZADA_DRIVE = "AI/ComfyUI" #@param {type:"string"}

#@markdown ---
#@markdown ### 🔄 Updates and Maintenance
ACTUALIZAR_COMFY = True #@param {type:"boolean"}
#@markdown > **Update:** Automatically downloads the latest version of ComfyUI from GitHub.

#@markdown ---
#@markdown ### 🧠 Model Installation and AI
#@markdown Select which models you want to pre-install to start working immediately.
INSTALAR_QWEN_VL = False #@param {type:"boolean"}
#@markdown > **Qwen VL/2509 Image Editing:** Installs the necessary models to use these Workflows.
#@markdown >
#@markdown > An advanced multimodal model capable of "understanding" images and editing them following complex instructions. Requires specific nodes that will be installed automatically.
#@markdown >
#@markdown > <font color='red'>Note: This model is very heavy, so Google Colab free machines usually cannot handle it (Stops due to lack of RAM).</font>

INSTALAR_SD15_BASE = False #@param {type:"boolean"}
#@markdown > **Stable Diffusion 1.5:** The gold standard for image generation. Includes the base model and a VAE to improve colors and faces.

INSTALAR_PROTOVISION_XL = False #@param {type:"boolean"}
INSTALAR_JUGGERNAUT_XL_HYPER = False #@param {type:"boolean"}
INSTALAR_JUGGERNAUT_X = False #@param {type:"boolean"}
INSTALAR_JUGGERNAUT_V8 = False #@param {type:"boolean"}

#@markdown ---
#@markdown ### 📦 Install Custom Models
#@markdown Enter the list of models in JSON format.
#@markdown > You can generate the JSON string with the correct format via the following Google Apps Script project:
#@markdown > ### [👉 Click HERE to open the tool](https://script.google.com/macros/s/AKfycbzmfmxfFRKvlnwlDuHNwOhWqEzYd90f1YKaQxSuFQ3o8HyQW_oHZZgPo9j4vUQlJ7ZI/exec)

MODELOS_PERSONALIZADOS = "[]" #@param {type:"string"}

#@markdown ---
#@markdown ### 🚀 Execution Configuration

USAR_LOWVRAM = False #@param {type:"boolean"}
#@markdown > **📉 Low VRAM:** Enable this if Colab closes unexpectedly or you run out of memory. Helps manage resources better.

USAR_CPU_ONLY = True #@param {type:"boolean"}
#@markdown > **🐌 CPU Only:** Enable this if you don't have access to a GPU. Generation will be extremely slow.

TIPO_TUNEL = "LOCALTUNNEL" #@param ["CLOUDFLARE", "LOCALTUNNEL"]
#@markdown > **🌐 Tunnel Type:** Select the connection method to access the web interface.

ARGUMENTOS_ADICIONALES = "" #@param {type:"string"}
#@markdown > **⌨️ Additional Arguments:** Insert extra commands to modify program behavior.

from google.colab import drive
from pathlib import Path
import os
import torch

if not torch.cuda.is_available():
    if not USAR_CPU_ONLY:
        print("\n" + "!"*60)
        print("❌ CRITICAL ERROR: NO GPU DETECTED")
        print("------------------------------------------------------------")
        print("ComfyUI requires a GPU to function. You are currently using CPU.")
        print("👉 Go to: Runtime > Change runtime type")
        print("👉 Select 'T4 GPU' in Hardware accelerator.")
        print("!"*60 + "\n")
        raise RuntimeError("You must select a GPU (T4) environment to continue.")
    else:
        print("\n⚠️ RUNNING IN CPU MODE: Image generation will be very slow.")

if MODO_DRIVE == "MONTAR":
  drive.mount('/content/drive', force_remount=True)
elif MODO_DRIVE == "DESMONTAR":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

WORKSPACE = '/content/ComfyUI'
if MODO_DRIVE == "MONTAR" and USAR_DRIVE_PARA_COMFY:
    WORKSPACE = "/content/drive/MyDrive/" + RUTA_PERSONALIZADA_DRIVE

if not os.path.exists(WORKSPACE):
    print("-= Initial setup ComfyUI =-")
    !git clone https://github.com/comfyanonymous/ComfyUI {WORKSPACE}
else:
    if ACTUALIZAR_COMFY:
        print("-= Updating ComfyUI =-")
        %cd {WORKSPACE}
        !git pull

%cd {WORKSPACE}

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}



%cd {WORKSPACE}
# Install Aria2
!apt-get -y install -qq aria2


from google.colab import userdata

CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
print("Loaded API key:", "✅" if CIVITAI_API_TOKEN else "❌ Not found")

# Install custom nodes

def install_custom_node(url):
  %cd {WORKSPACE}/custom_nodes
  !git clone {url}

def downloadModel(url, filename = None):
  if 'huggingface.co' in url:
    if filename is None:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    if filename:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN}


# install custom nodes
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager.git')
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')


# download models
%cd ./models/checkpoints

# CyberRealistic Pony
downloadModel('https://civitai.com/api/download/models/2071650')

# Protovision XL
if INSTALAR_PROTOVISION_XL:
    downloadModel('https://civitai.com/api/download/models/201514')
# Juggernaut XL Hyper
if INSTALAR_JUGGERNAUT_XL_HYPER:
    downloadModel('https://civitai.com/api/download/models/471120')
# Juppernaut X
if INSTALAR_JUGGERNAUT_X:
    downloadModel('https://civitai.com/api/download/models/456194')
# Juggernaut v8
if INSTALAR_JUGGERNAUT_V8:
    downloadModel('https://civitai.com/api/download/models/288982')

%cd {WORKSPACE}



if INSTALAR_QWEN_VL:
    # 1. Configuración de carpetas y descargas de Qwen
    # ----------------------------------------------------------------

    # VAE
    %cd {WORKSPACE}/models/vae
    downloadModel('https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/vae/qwen_image_vae.safetensors')

    # DIFFUSION MODELS (Unet)
    # Nota: Si la carpeta no existe, la creamos
    !mkdir -p {WORKSPACE}/models/diffusion_models
    %cd {WORKSPACE}/models/diffusion_models
    downloadModel('https://huggingface.co/Comfy-Org/Qwen-Image-Edit_ComfyUI/resolve/main/split_files/diffusion_models/qwen_image_edit_fp8_e4m3fn.safetensors')

    # TEXT ENCODERS
    !mkdir -p {WORKSPACE}/models/text_encoders
    %cd {WORKSPACE}/models/text_encoders
    downloadModel('https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/text_encoders/qwen_2.5_vl_7b_fp8_scaled.safetensors')

    # LORAS
    %cd {WORKSPACE}/models/loras
    downloadModel('https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Edit-Lightning-4steps-V1.0-bf16.safetensors')

    # 2. Instalación del Nodo Personalizado (Obligatorio para que funcione el workflow)
    # ----------------------------------------------------------------
    %cd {WORKSPACE}/custom_nodes
    # Clonamos el repositorio oficial que maneja estos modelos de Qwen
    !git clone https://github.com/Comfy-Org/ComfyUI-Qwen-VL-API.git 2>/dev/null || echo "🟢 Qwen/2095 Image Editing - Installation Finished"

    # Volver al inicio
    %cd {WORKSPACE}

if INSTALAR_SD15_BASE:
    # --- Descarga de Modelos SD 1.5 (Optimizados) ---

    # SD1.5 Pruned (Versión Safetensors - Más rápida y segura)
    %cd {WORKSPACE}/models/checkpoints
    downloadModel('https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors')

    # AbyssOrangeMix2 (Anime Style)
    downloadModel('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors')

    # VAE Standard (Mejora el color y detalles en SD 1.5)
    %cd {WORKSPACE}/models/vae
    downloadModel('https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors')

    %cd {WORKSPACE}

if MODELOS_PERSONALIZADOS:
    import json
    print("⬇️ Downloading custom models...")
    try:
        custom_models = json.loads(MODELOS_PERSONALIZADOS)
        if isinstance(custom_models, list):
            for model in custom_models:
                path_str = model.get("ruta", "").strip()
                url = model.get("link", "").strip()

                if path_str and url:
                    if "/" in path_str:
                        folder_part, filename_part = path_str.rsplit("/", 1)
                        folder_part = folder_part.strip()
                        filename_part = filename_part.strip()

                        target_path = f"{WORKSPACE}/models/{folder_part}"
                        !mkdir -p {target_path}
                        %cd {target_path}
                        downloadModel(url, filename_part)
                    else:
                        print(f"⚠️ Incorrect format in path: {path_str}")
    except json.JSONDecodeError:
        print("⚠️ Error: MODELOS_PERSONALIZADOS is not a valid JSON.")

    %cd {WORKSPACE}


!ls -al ./models/checkpoints/


if TIPO_TUNEL == "CLOUDFLARE":
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb
elif TIPO_TUNEL == "LOCALTUNNEL":
    !npm install -g localtunnel



# --- CREACIÓN DEL SCRIPT LANZADOR PERSISTENTE ---
%cd {WORKSPACE}

comfy_args = ["python", "main.py", "--dont-print-server"]
if USAR_CPU_ONLY:
    comfy_args.append("--cpu")
if USAR_LOWVRAM:
    comfy_args.append("--lowvram")
if ARGUMENTOS_ADICIONALES:
    comfy_args.extend(ARGUMENTOS_ADICIONALES.split())

launcher_script = f"""
import subprocess
import threading
import time
import socket
import os
import urllib.request

def launch_tunnel(port):
    print("\\n[Tunnel] Waiting for ComfyUI to start...")
    while True:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        if sock.connect_ex(('127.0.0.1', port)) == 0:
            sock.close()
            break
        sock.close()
        time.sleep(1)

    tunnel_type = "{TIPO_TUNEL}"

    if tunnel_type == "CLOUDFLARE":
        proc = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{{}}".format(port)],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        for line in proc.stderr:
            if "trycloudflare.com " in line:
                url = line.split("http")[1].strip()
                print("\\n" + "="*50)
                print(f"🔗 ACCESS URL: http{{url}}")
                print("="*50 + "\\n")

    elif tunnel_type == "LOCALTUNNEL":
        try:
            ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\\n")
            print("\\n" + "="*50)
            print(f"🔐 LOCALTUNNEL PASSWORD/IP: {{ip}}")
            print("="*50 + "\\n")
        except:
            pass

        proc = subprocess.Popen(["lt", "--port", "{{}}".format(port)],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        for line in proc.stdout:
            print(line, end='')
            if "your url is" in line:
                url = line.split("is:")[1].strip()
                print("\\n" + "="*50)
                print(f"🔗 ACCESS URL: {{url}}")
                print("="*50 + "\\n")

if __name__ == "__main__":
    threading.Thread(target=launch_tunnel, args=(8188,), daemon=True).start()
    subprocess.run({comfy_args})
"""

with open("colab_launcher.py", "w") as f:
    f.write(launcher_script)

print("\n🚀 Starting ComfyUI...")
# Ejecutar en segundo plano con nohup para liberar la celda
!nohup python -u colab_launcher.py > comfy.log 2>&1 &

print("⏳ Waiting for Cloudflare URL (this may take a few seconds)...")
import time
found = False
file_pos = 0
while not found:
    if os.path.exists("comfy.log"):
        with open("comfy.log", "r") as f:
            f.seek(file_pos)
            chunk = f.read()
            if chunk:
                print(chunk, end='', flush=True)
                file_pos = f.tell()
                if "trycloudflare.com" in chunk or "your url is" in chunk:
                    for line in chunk.splitlines():
                        if "trycloudflare.com" in line:
                            url = line.split("http")[1].strip()
                            print(f"\n✅ ACCESS URL: http{url}")
                            print("⚠️ The cell will close, but ComfyUI continues running in the background.")
                            found = True
                            break
                        elif "your url is" in line:
                            url = line.split("is:")[1].strip()
                            print(f"\n✅ ACCESS URL: {url}")
                            print("⚠️ The cell will close, but ComfyUI continues running in the background.")
                            found = True
                            break
    time.sleep(2)

In [ ]:
#@title # 📦 Install Custom Models Post-Execution
#@markdown ---
#@markdown ## If you need to install models after the main execution, you can do it from here:
#@markdown Enter the list of models in JSON format in the box below, after placing the JSON line execute the cell.
#@markdown > You can generate the JSON string with the correct format via the following Google Apps Script project:
#@markdown > ### [👉 Click HERE to open the tool](https://script.google.com/macros/s/AKfycbzmfmxfFRKvlnwlDuHNwOhWqEzYd90f1YKaQxSuFQ3o8HyQW_oHZZgPo9j4vUQlJ7ZI/exec)

MODELOS_PERSONALIZADOS = "[{\"id\":1,\"ruta\":\"vae/ae.safetensors\",\"link\":\"https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors\",\"workflow\":\"image_omnigen2_image_edit\"},{\"id\":2,\"ruta\":\"text_encoders/qwen_2.5_vl_fp16.safetensors\",\"link\":\"https://huggingface.co/Comfy-Org/Omnigen2_ComfyUI_repackaged/resolve/main/split_files/text_encoders/qwen_2.5_vl_fp16.safetensors\",\"workflow\":\"image_omnigen2_image_edit\"},{\"id\":3,\"ruta\":\"diffusion_models/omnigen2_fp16.safetensors\",\"link\":\"https://huggingface.co/Comfy-Org/Omnigen2_ComfyUI_repackaged/resolve/main/split_files/diffusion_models/omnigen2_fp16.safetensors\",\"workflow\":\"image_omnigen2_image_edit\"}]" #@param {type:"string"}
#@markdown ---
#@markdown <font color='#888'>Note: Make sure you have executed the main cell first to define the `WORKSPACE` path.</font>

import json
import os
from google.colab import userdata

def descargar_modelos_custom():
    if not MODELOS_PERSONALIZADOS or MODELOS_PERSONALIZADOS == "[]":
        print("ℹ️ No models entered for download.")
        return

    # Try to load Civitai token if it exists
    try:
        CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
    except:
        CIVITAI_API_TOKEN = ""
        print("⚠️ Civitai Token not configured in 'Secrets'.")

    def download_logic(url, filename=None):
        auth_token = f"?token={CIVITAI_API_TOKEN}" if CIVITAI_API_TOKEN and "civitai.com" in url else ""
        output_cmd = f"-o {filename}" if filename else ""

        # We use aria2 for fast and resilient downloads
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}{auth_token}" {output_cmd}

    print("⬇️ Starting custom model download...")

    try:
        custom_models = json.loads(MODELOS_PERSONALIZADOS)
        if isinstance(custom_models, list):
            for model in custom_models:
                path_str = model.get("ruta", "").strip()
                url = model.get("link", "").strip()

                if path_str and url:
                    # Separate folder and filename
                    if "/" in path_str:
                        folder_part, filename_part = path_str.rsplit("/", 1)
                    else:
                        folder_part = "checkpoints" # Default folder
                        filename_part = path_str

                    # Build full path
                    target_path = os.path.join(WORKSPACE, "models", folder_part)
                    os.makedirs(target_path, exist_ok=True)

                    %cd {target_path}
                    print(f"📥 Downloading: {filename_part} in models/{folder_part}")
                    download_logic(url, filename_part)
                else:
                    print(f"⚠️ Skipping invalid entry: {model}")

        print("\n✅ Download process finished!")
        %cd {WORKSPACE}

    except json.JSONDecodeError:
        print("❌ Error: The content of MODELOS_PERSONALIZADOS is not a valid JSON.")
    except NameError:
        print("❌ Error: The 'WORKSPACE' variable is not defined. Run the installation cell first.")

descargar_modelos_custom()